In [5]:
# Load
import warnings
import pandas as pd

warnings.filterwarnings(action="ignore")

from model_tool.technical._preproc import OhlcvPreproc
from model_tool.technical.highlowbased._preproc import HighLowBasedPositionGenerator
from model_tool.technical.pricebased._preproc import PriceBasedPositionGenerator
from model_tool.technical._model import PositionLimitModel, PositionRankIndexModel

In [6]:
kosdaq_ohlcv_df = pd.read_csv("./data/fdr/kosdaq_ohlcv_df.csv", index_col=0)
kospi_ohlcv_df = pd.read_csv("./data/fdr/kospi_ohlcv_df.csv", index_col=0)

In [7]:
ohlcv_preproc = OhlcvPreproc(kospi_ohlcv_df, kosdaq_ohlcv_df)
ohlcv_df = ohlcv_preproc()

In [8]:
pb_pg = PriceBasedPositionGenerator()
hlb_pg = HighLowBasedPositionGenerator()

pri_model = PositionRankIndexModel()
pl_model = PositionLimitModel()

In [53]:
fundamental_results = pd.read_csv("./fundamental_result.csv", index_col=0)
fundamental_results["stock_code"] = fundamental_results["stock_code"].apply(
    lambda x: str(x).zfill(6)
)
using_stock_codes = sorted(set(fundamental_results["stock_code"]))

In [23]:
analysis_dict = dict()

time_size = 60
position_size = 100

for stock_code in using_stock_codes:
    try:
        _ohlcv_df = ohlcv_df[ohlcv_df["stock_code"] == stock_code]
        pb_tdvp = pb_pg.get_time_dependent_volume_position(
            prices=_ohlcv_df["close"],
            volumes=_ohlcv_df["volume"],
            time_size=time_size,
            position_size=position_size,
        )
        hlb_tdvp = hlb_pg.get_time_dependent_volume_position(
            highs=_ohlcv_df["high"],
            lows=_ohlcv_df["low"],
            volumes=_ohlcv_df["volume"],
            time_size=time_size,
            position_size=position_size,
        )
        analysis_dict[stock_code] = {
            "price_based_low_high": pl_model.calc(pb_tdvp),
            "high_low_based_low_high": pl_model.calc(hlb_tdvp),
            "price_based_index": pri_model.calc(
                _ohlcv_df["close"].iloc[-1], pb_tdvp
            ),
            "high_low_based_index": pri_model.calc(
                _ohlcv_df["close"].iloc[-1], hlb_tdvp
            ),
        }
    except:
        print(stock_code)

008370
012790
105740


In [29]:
analysis_df = pd.DataFrame(analysis_dict).T

In [41]:
latest_price_dict = ohlcv_df.groupby("stock_code").tail(1).set_index("stock_code")['close']

In [42]:
analysis_df['price'] = analysis_df.index.map(latest_price_dict)

In [45]:
def in_low_high(low_high, price):
    low, high = low_high
    if (low < price) and (price < high):
        return True
    else:
        return False

In [49]:
selected = analysis_df[
    (
        analysis_df.apply(
            lambda x: in_low_high(x["price_based_low_high"], x["price"]),
            axis=1,
        )
    )
    & (
        analysis_df.apply(
            lambda x: in_low_high(x["high_low_based_low_high"], x["price"]),
            axis=1,
        )
    )
]

In [51]:
selected.index

Index(['000500', '002460', '013700', '015590', '034590', '045100', '053620',
       '084870', '250000'],
      dtype='object')

In [54]:
fundamental_results[fundamental_results["stock_code"].isin(selected.index)]

,stock_code,stock_name,stock_share
580,045100,한양이엔지,18000000
686,000500,가온전선,7357946
1387,034590,인천도시가스,4374754
1444,015590,KIB플러그에너지,236981544
1470,002460,화성산업,10150000
1777,250000,보라티알,6750733
1817,013700,까뮤이앤씨,45152280
1958,053620,태양,8600000
2318,084870,TBH글로벌,20856819
